In [69]:
import os
import cv2
import pickle
import mediapipe as mp
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [70]:
# Load
with open('x_train.pkl', 'rb') as file:
    x_train = pickle.load(file)

with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)
with open('x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)

with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)


In [71]:

class CNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CNN_LSTM, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # cnn takes input of shape (batch_size, channels, seq_len)
        # print("x equals: ",x)
        # x = x[0][0][0]
        # x = torch.tensor(x)
        x = x.permute(0, 2, 1)
        out = self.cnn(x)
        # lstm takes input of shape (batch_size, seq_len, input_size)
        out = out.permute(0, 2, 1)
        out, _ = self.lstm(out)
        out = self.fc(out[:, -1, :])
        return out


In [72]:
from torch.nn.utils.rnn import pad_sequence

# Assuming x_train is a list of tensors with variable lengths
# Convert x_train to a list of torch tensors

x_train = [torch.tensor(row, dtype=torch.float32) for row in x_train]

# Pad sequences to make them the same length

x_train = pad_sequence(x_train, batch_first=True, padding_value=0)

# Now x_train_padded contains the padded sequences with the same length

In [73]:
x_train


tensor([[[[[[0.2747, 0.9711, 0.1519],
            [0.2654, 0.9813, 0.1329],
            [0.2694, 0.9606, 0.0967],
            ...,
            [0.5255, 0.7296, 0.1994],
            [0.7167, 0.6448, 0.3350],
            [0.7217, 0.6438, 0.5097]]]],



         [[[[0.2810, 0.9666, 0.2426],
            [0.2653, 0.9803, 0.1755],
            [0.2703, 0.9587, 0.2249],
            ...,
            [0.5628, 0.6424, 0.1664],
            [0.7190, 0.6422, 0.2386],
            [0.7252, 0.6391, 0.5088]]]],



         [[[[0.2779, 0.9705, 0.2488],
            [0.2625, 0.9835, 0.1559],
            [0.2644, 0.9656, 0.2193],
            ...,
            [0.4755, 0.7968, 0.1343],
            [0.6516, 0.8479, 0.0384],
            [0.7282, 0.6428, 0.5025]]]],



         ...,



         [[[[0.1707, 0.3383, 0.2192],
            [0.1525, 0.3526, 0.4098],
            [0.1590, 0.3270, 0.3003],
            ...,
            [0.5264, 0.2949, 0.4614],
            [0.6274, 0.4258, 0.1915],
            [0.6277, 0.

In [74]:

# Initialize the model, loss function, and optimizer
input_size = 3  # Set your input size
hidden_size = 3  # Set your hidden size
num_layers = 5  # Set your number of layers
num_classes = len(np.unique(y_train))  # Set your number of classes
model = CNN_LSTM(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [75]:
# Training loop
num_epochs = 5  # Set your number of epochs
batch_size = 5  # Set your batch size


In [76]:

for epoch in range(num_epochs):
    for i in range(0, len(x_train), batch_size):
        inputs = x_train[i:i + batch_size]
        labels = y_train[i:i + batch_size]

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(x_train) // batch_size}], Loss: {loss.item()}')

print('Training finished.')

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 6 is not equal to len(dims) = 3